In [1]:
import pandas as pd

dt = pd.read_csv("./prometheus_data.csv")
dt

,__name__,container,endpoint,instance,job,namespace,pod,service,timestamp,value,metrics_path,node
0,up,alertmanager,http-web,10.46.0.7:9093,prometheus-kube-prometheus-alertmanager,prometheus,alertmanager-prometheus-kube-prometheus-alertm...,prometheus-kube-prometheus-alertmanager,2024-04-17 10:51:34,1,NaN,NaN
1,up,alertmanager,http-web,10.46.0.7:9093,prometheus-kube-prometheus-alertmanager,prometheus,alertmanager-prometheus-kube-prometheus-alertm...,prometheus-kube-prometheus-alertmanager,2024-04-17 10:51:49,1,NaN,NaN
2,up,alertmanager,http-web,10.46.0.7:9093,prometheus-kube-prometheus-alertmanager,prometheus,alertmanager-prometheus-kube-prometheus-alertm...,prometheus-kube-prometheus-alertmanager,2024-04-17 10:52:04,1,NaN,NaN
3,up,alertmanager,http-web,10.46.0.7:9093,prometheus-kube-prometheus-alertmanager,prometheus,alertmanager-prometheus-kube-prometheus-alertm...,prometheus-kube-prometheus-alertmanager,2024-04-17 10:52:19,1,NaN,NaN
4,up,alertmanager,http-web,10.46.0.7:9093,prometheus-kube-prometheus-alertmanager,prometheus,alertmanager-prometheus-kube-prometheus-alertm...,prometheus-kube-prometheus-alertmanager,2024-04-17 10:52:34,1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
11008,up,NaN,https-metrics,192.168.56.12:10250,kubelet,kube-system,NaN,prometheus-kube-prometheus-kubelet,2024-04-17 12:44:04,1,/metrics/probes,w2
11009,up,NaN,https-metrics,192.168.56.12:10250,kubelet,kube-system,NaN,prometheus-kube-prometheus-kubelet,2024-04-17 12:44:19,1,/metrics/probes,w2
11010,up,NaN,https-metrics,192.168.56.12:10250,kubelet,kube-system,NaN,prometheus-kube-prometheus-kubelet,2024-04-17 12:44:34,1,/metrics/probes,w2
11011,up,NaN,https-metrics,192.168.56.12:10250,kubelet,kube-system,NaN,prometheus-kube-prometheus-kubelet,2024-04-17 12:44:49,1,/metrics/probes,w2


In [10]:
from prometheus_api_client import PrometheusConnect
# prom = PrometheusConnect(url ="http://localhost:9090", disable_ssl=True)
prom = PrometheusConnect(url ="http://192.168.56.10:30090", disable_ssl=True)
import datetime
from datetime import datetime, timedelta  
import pandas as pd


In [6]:
# !pip install prometheus-api-client
!pip install pandas


In [11]:


# Instant Query
result = prom.custom_query(query="node_cpu_seconds_total")
# print(result)
# Range Query
result_range = prom.custom_query_range(
    query="node_cpu_seconds_total",
    start_time= datetime.now() - timedelta(hours=2),
    end_time=datetime.now(),
    step='15s'
)
print(result_range)

[{'metric': {'__name__': 'node_cpu_seconds_total', 'container': 'node-exporter', 'cpu': '0', 'endpoint': 'http-metrics', 'instance': '192.168.56.10:9100', 'job': 'node-exporter', 'mode': 'idle', 'namespace': 'prometheus', 'pod': 'prometheus-prometheus-node-exporter-gh7qx', 'service': 'prometheus-prometheus-node-exporter'}, 'values': [[1713359613, '7359.27'], [1713359628, '7388.17'], [1713359643, '7388.17'], [1713359658, '7417.4'], [1713359673, '7417.4'], [1713359688, '7446.49'], [1713359703, '7446.49'], [1713359718, '7475.77'], [1713359733, '7475.77'], [1713359748, '7505.04'], [1713359763, '7505.04'], [1713359778, '7534.18'], [1713359793, '7534.18'], [1713359808, '7563.14'], [1713359823, '7563.14'], [1713359838, '7592.33'], [1713359853, '7592.33'], [1713359868, '7621.42'], [1713359883, '7621.42'], [1713359898, '7650.6'], [1713359913, '7650.6'], [1713359928, '7679.82'], [1713359943, '7679.82'], [1713359958, '7709.14'], [1713359973, '7709.14'], [1713359988, '7738.27'], [1713360003, '7738

In [12]:
import json

# Assuming 'result_range' is your data loaded from Prometheus
# result_range = [...]  # This should be replaced with your actual data query from Prometheus

# Define the path to save the JSON file
file_path = 'prometheus_data_node_cpu_seconds_total.json'

# Save the data to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(result_range, json_file, indent=4)

print(f"Data has been saved to {file_path}.")


Data has been saved to prometheus_data_node_cpu_seconds_total.json.


In [13]:


# result_range = prom.custom_query_range(
#     query='100 - (avg by (instance) (rate(node_cpu_seconds_total{mode="idle"}[1m])) * 100)',
#     start_time=datetime.now() - timedelta(hours=2),
#     end_time=datetime.now(),
#     step='15s'
# )
# print(result_range)

# Creating a list to hold data for DataFrame
data = []
for result in result_range:
    for value in result['values']:
        timestamp = value[0]
        metric_value = value[1]
        metric_info = result['metric']
        data.append({**metric_info, 'timestamp': datetime.fromtimestamp(float(timestamp)), 'value': metric_value})

# Convert list to DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('prometheus_data2.csv', index=False)

# Optional: Save to other formats
# Save to Excel
# df.to_excel('prometheus_data.xlsx', index=False)


# Display the DataFrame
# print(df.head())


In [14]:
from prometheus_api_client import PrometheusConnect

# Setup connection (ensure the URL is correct for your Prometheus server)
prom = PrometheusConnect(url ="http://192.168.56.10:30090", disable_ssl=True)

# prom = PrometheusConnect(url ="http://your-prometheus-server-url", disable_ssl=True)

# Fetch all metric names
all_metrics = prom.all_metrics()

# Print all metric names
print("Available Metrics:")
for metric in all_metrics:
    print(metric)


Available Metrics:
:node_memory_MemAvailable_bytes:sum
ALERTS
ALERTS_FOR_STATE
aggregator_discovery_aggregation_count_total
aggregator_unavailable_apiservice
alertmanager_alerts
alertmanager_alerts_invalid_total
alertmanager_alerts_received_total
alertmanager_build_info
alertmanager_cluster_enabled
alertmanager_config_hash
alertmanager_config_last_reload_success_timestamp_seconds
alertmanager_config_last_reload_successful
alertmanager_dispatcher_aggregation_groups
alertmanager_dispatcher_alert_processing_duration_seconds_count
alertmanager_dispatcher_alert_processing_duration_seconds_sum
alertmanager_http_concurrency_limit_exceeded_total
alertmanager_http_request_duration_seconds_bucket
alertmanager_http_request_duration_seconds_count
alertmanager_http_request_duration_seconds_sum
alertmanager_http_requests_in_flight
alertmanager_http_response_size_bytes_bucket
alertmanager_http_response_size_bytes_count
alertmanager_http_response_size_bytes_sum
alertmanager_inhibition_rules
alertmanag

In [15]:
from prometheus_api_client import PrometheusConnect

# Setup connection (ensure the URL is correct for your Prometheus server)
# prom = PrometheusConnect(url ="http://your-prometheus-server-url", disable_ssl=True)
prom = PrometheusConnect(url ="http://192.168.56.10:30090", disable_ssl=True)


# Fetch all metric names
all_metrics = prom.all_metrics()

# Define the file path where you want to save the metrics
file_path = 'prometheus_metrics.txt'

# Write all metrics to the text file, one per line
with open(file_path, 'w') as file:
    for metric in all_metrics:
        file.write(metric + '\n')

print(f"Metrics have been saved to {file_path}.")


Metrics have been saved to prometheus_metrics.txt.


In [ ]:
import json
from prometheus_api_client import PrometheusConnect
from datetime import datetime, timedelta

# Setup connection
# prom = PrometheusConnect(url="http://your-prometheus-server-url", disable_ssl=True)
prom = PrometheusConnect(url ="http://192.168.56.10:30090", disable_ssl=True)


# Fetch all metric names
all_metrics = prom.all_metrics()

# Define a starting point for data collection (this might need adjustment)
# Here, it's arbitrarily set to 5 years ago. Adjust based on your actual data collection start.
start_time = datetime.now() - timedelta(days=5*365)

# Current time for the end of data collection
end_time = datetime.now()

# Step size for queries (e.g., "15m" for 15 minutes)
step = '15m'

for metric in all_metrics:
    # Perform a range query for each metric
    result_range = prom.custom_query_range(
        query=metric,
        start_time=start_time,
        end_time=end_time,
        step=step
    )

    # File to save the results
    file_path = f'{metric}.json'
    with open(file_path, 'w') as file:
        json.dump(result_range, file, indent=4)

    print(f"Data for {metric} has been saved to {file_path}.")


In [3]:
import json
import gzip
from prometheus_api_client import PrometheusConnect
from datetime import datetime, timedelta

# Setup connection
# prom = PrometheusConnect(url="http://your-prometheus-server-url", disable_ssl=True)
prom = PrometheusConnect(url ="http://192.168.56.10:30090", disable_ssl=True)

# Fetch all metric names
all_metrics = prom.all_metrics()

# Define a starting point for data collection
start_time = datetime.now() - timedelta(days=1)  # Example: 5 years ago
end_time = datetime.now()  # Current time
step = '15s'  # Step size

for metric in all_metrics:
    # Perform a range query for each metric
    result_range = prom.custom_query_range(
        query=metric,
        start_time=start_time,
        end_time=end_time,
        step=step
    )

    # Define file path with .gz extension
    file_path = f'./data/{metric.replace("/", "_")}.json.gz'  # Replace '/' with '_' to avoid path issues

    # Write data to a gzipped JSON file
    with gzip.open(file_path, 'wt', encoding='utf-8') as file:
        json.dump(result_range, file, indent=4)

    print(f"Data for {metric} has been saved to {file_path}.")


Data for :node_memory_MemAvailable_bytes:sum has been saved to ./data/:node_memory_MemAvailable_bytes:sum.json.gz.
Data for ALERTS has been saved to ./data/ALERTS.json.gz.
Data for ALERTS_FOR_STATE has been saved to ./data/ALERTS_FOR_STATE.json.gz.
Data for aggregator_discovery_aggregation_count_total has been saved to ./data/aggregator_discovery_aggregation_count_total.json.gz.
Data for aggregator_unavailable_apiservice has been saved to ./data/aggregator_unavailable_apiservice.json.gz.
Data for alertmanager_alerts has been saved to ./data/alertmanager_alerts.json.gz.
Data for alertmanager_alerts_invalid_total has been saved to ./data/alertmanager_alerts_invalid_total.json.gz.
Data for alertmanager_alerts_received_total has been saved to ./data/alertmanager_alerts_received_total.json.gz.
Data for alertmanager_build_info has been saved to ./data/alertmanager_build_info.json.gz.
Data for alertmanager_cluster_enabled has been saved to ./data/alertmanager_cluster_enabled.json.gz.
Data for